In [1]:
"""Script simplificado para descargar facturas del dataset de Hugging Face"""

import pandas as pd
import json
import io
from pathlib import Path
from PIL import Image
from huggingface_hub import hf_hub_download
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter


def create_pdf_from_image(image_path, pdf_path):
    """Crear PDF simple desde imagen"""
    image = Image.open(image_path)
    img_width, img_height = image.size
    
    # Crear PDF
    c = canvas.Canvas(str(pdf_path), pagesize=letter)
    page_width, page_height = letter
    
    # Calcular dimensiones manteniendo aspecto
    aspect = img_height / float(img_width)
    display_width = page_width - 40
    display_height = display_width * aspect
    
    if display_height > page_height - 40:
        display_height = page_height - 40
        display_width = display_height / aspect
    
    # Dibujar imagen
    c.drawImage(str(image_path), 20, page_height - display_height - 20,
               width=display_width, height=display_height)
    c.save()


def main():
    print("=" * 60)
    print("📄 DESCARGADOR DE FACTURAS")
    print("=" * 60)
    
    # Configuración
    repo_id = "katanaml-org/invoices-donut-data-v1"
    filename = "data/train-00000-of-00001-a5c51039eab2980a.parquet"
    facturas_dir = Path('../facturas_dataset')
    facturas_dir.mkdir(parents=True, exist_ok=True)
    
    # Descargar dataset
    print(f"\n📥 Descargando dataset de Hugging Face...")
    print(f"   Repositorio: {repo_id}")
    
    try:
        parquet_file = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            repo_type="dataset"
        )
        print(f"✅ Archivo descargado")
        
        # Leer datos
        df = pd.read_parquet(parquet_file, engine='fastparquet')
        print(f"✅ Datos cargados: {len(df)} facturas")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return
    
    # Procesar facturas
    num_samples = min(50, len(df))
    print(f"\n💾 Procesando {num_samples} facturas...\n")
    
    for i in range(num_samples):
        try:
            row = df.iloc[i]
            
            if 'image.bytes' not in row:
                continue
            
            image_bytes = row['image.bytes']
            image = Image.open(io.BytesIO(image_bytes))
            
            # Guardar PNG
            png_path = facturas_dir / f"factura_{i+1}.png"
            image.save(png_path, 'PNG')
            print(f"[{i+1}/{num_samples}] ✅ Imagen: {png_path.name}")
            
            # Crear PDF simple
            pdf_path = facturas_dir / f"factura_{i+1}.pdf"
            create_pdf_from_image(png_path, pdf_path)
            print(f"           ✅ PDF: {pdf_path.name}")
            
            # Guardar metadata
            metadata = {
                "numero": i + 1,
                "ground_truth": row.get('ground_truth', ''),
                "image_path": row.get('image.path', ''),
                "archivos": {
                    "imagen": f"factura_{i+1}.png",
                    "pdf": f"factura_{i+1}.pdf"
                }
            }
            
            metadata_path = facturas_dir / f"factura_{i+1}_metadata.json"
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            print(f"           ✅ Metadata guardada\n")
            
        except Exception as e:
            print(f"❌ Error procesando factura {i+1}: {e}\n")
            continue
    
    print("\n" + "=" * 60)
    print("🎉 PROCESO COMPLETADO")
    print("=" * 60)
    print(f"📁 Facturas guardadas en: {facturas_dir.absolute()}")
    print(f"\n💡 NOTA:")
    print(f"   - Los PDFs contienen imágenes (no texto seleccionable)")
    print(f"   - El backend usará Vision API automáticamente para procesarlos")
    print(f"   - También puedes usar los archivos PNG directamente")


if __name__ == "__main__":
    main()

📄 DESCARGADOR DE FACTURAS

📥 Descargando dataset de Hugging Face...
   Repositorio: katanaml-org/invoices-donut-data-v1
✅ Archivo descargado
✅ Datos cargados: 425 facturas

💾 Procesando 50 facturas...

[1/50] ✅ Imagen: factura_1.png
           ✅ PDF: factura_1.pdf
           ✅ Metadata guardada

[2/50] ✅ Imagen: factura_2.png
           ✅ PDF: factura_2.pdf
           ✅ Metadata guardada

[3/50] ✅ Imagen: factura_3.png
           ✅ PDF: factura_3.pdf
           ✅ Metadata guardada

[4/50] ✅ Imagen: factura_4.png
           ✅ PDF: factura_4.pdf
           ✅ Metadata guardada

[5/50] ✅ Imagen: factura_5.png
           ✅ PDF: factura_5.pdf
           ✅ Metadata guardada

[6/50] ✅ Imagen: factura_6.png
           ✅ PDF: factura_6.pdf
           ✅ Metadata guardada

[7/50] ✅ Imagen: factura_7.png
           ✅ PDF: factura_7.pdf
           ✅ Metadata guardada

[8/50] ✅ Imagen: factura_8.png
           ✅ PDF: factura_8.pdf
           ✅ Metadata guardada

[9/50] ✅ Imagen: factura_9.png
       